In [1]:
import svmMLiA

In [2]:
dataArr, labelArr = svmMLiA.loadDataSet('testSet.txt')
labelArr

[-1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 1.0,
 -1.0,
 1.0,
 1.0,
 1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0]

In [3]:
def generateI(alphas, labelMat, dataMatrix, i, b):
    # 如果这一轮要更新alpha_i，先把alpha_i对应的样本信息都准备好
    I = generateNode(alphas, labelMat, dataMatrix, i, b)
    # condition to choose i: error is big enough
    if(not choose(I, toler, C)):
        raise Exception('error not big enough')
    return I

In [4]:
def generateNode(dataMatrix, labelMat, alphas, index, b):
    return {'index':index, # 更新第几个alpha
             'x':dataMatrix[index,:], # 对应的数据
             'y':labelMat[index],     # 对应的标签
             'a':alphas[index],       # 当前的alpha值
            #E: error between predition class and real class
            # 基于当前alpha对第index个数据的预测分类与该数据的真实分类做比较
             'E':E(alphas, labelMat, dataMatrix, index, b)
           }

SVM对数据点$x_i$的预测值为：  
$$
f(x_i) = (\sum_{i=1}^{M}a_iy_i) \cdot K<x, x_i> + b \tag{1}
$$
记$E_i$为SVM预测值与真实值的误差：  
$$
E_i = f(x_i) - y_i \tag{2}
$$

In [25]:
def E(alphas, labelMat, dataMatrix, i, b):
    # fx: the predition of the class
    fXi = float(np.multiply(alphas, labelMat).T*(dataMatrix*dataMatrix[i,:].T)) + b
    # E: error between predition class and real class
    Ei = fXi - labelMat[i]
    return Ei

In [26]:
# If this error is large, then the alpha corresponding to this data instance can be optimized.
# ???
def choose(I, toler, C):
    return ((I['y']*I['E'] < - toler) and (I['a'] < C)) or ((I['y']*I['E'] > toler) and (I['a'] > 0))

In [27]:
def generateJ(alphas, labelMat, dataMatrix, i, b):
    # select j randomly
    j = selectJrand(i, m)
    return generateNode(alphas, labelMat, dataMatrix, j, b)

In [28]:
# i: the index of first alpha
# m: the total number of alphas
# choose a random valuse which is not equal to i
def selectJrand(i, m):
    j = i
    while (j==i):
        j = int(np.random.uniform(0, m))
    return j

目标函数：

$$
W(a) = \sum_{i=1}^{M}a - \frac{1}{2}\sum_{i,j=1}^{M}y_iy_ja_ia_jK_{i,j} \tag{3} \\
$$
约束条件：

$$
0 \le a \le C \tag {4}
$$
$$
\sum_{i=1}^{M}a_iy_i = 0 \tag {5} 
$$

In [29]:
# 更新alpha_i, alpha_j和b，使得目标函数进一步变大
def update(I, J, C):
    # 根据公式先更新alpha_j
    update_alpha_j(I, J, C)                    
    # change alpha_i as alpha_j changed
    update_alpha_i(I, J)
    # 当更新了一对a_i,a_j之后，需要重新计算b。
    b = update_b(b, I, J, C)
    return I['a'], J['a'], b

假设现在要更新的是$a_1$和$a_2$,即代码中的I和J，其它alpha都当作常数  
根据约束条件$\sum_{i=1}^M a_iy_i =0$可以得到a_1和a_2的关系：  
$$
a_1y_1 + a_2y_2 = - \sum_{i=3}^{M}a_iy_i = \zeta \tag{6}
$$
公式（6）左边两边同时乘以$y_1$，得：  
$$
a_1 = \zeta y_1 - a_2y_2y_1 \tag{7}
$$
其中$\zeta$代表一个常数。  
目标函数（3）中也把变量$a_1$、$a_2$和常量其它$a$分开，则写成  
$$
W(a_1, a_2) = a_1 + a_2 - \frac{1}{2}K_{1,1}y_1^2a_1^2 - \frac{1}{2}K_{2,2}y_2^2a_2^2 - K_{1,2}y_1y_2a_1a_2 - y_1a_1\sum_{i=3}^{M}a_iy_iK_{i,1} - y_2a_2\sum_{i=3}^{M}a_iy_iK_{i,2} + C\tag {8}
$$
其中$C$是一个与$a_1$、$a_2$无关的常数  
把公式（7）代码公式（8）中，同时为了简单公式（8）的写法，令：  
$$
v_1 = \sum_{i=3}^{M}a_iy_iK_{i,1} \\
v_2 = \sum_{i=3}^{M}a_iy_iK_{i,2} \tag {9}
$$
最终公式（8）变成了公式（10）  
$$
W(a_2) = - \frac{1}{2}K_{1,1}(\zeta - a_2y_2)^2 - \frac{1}{2}K_{2,2}a_2^2 - y_2(\zeta-a_2y_2)a_2K_{1,2} - v_1(\zeta-a_2y_2)-v_2y_2a_2+a_1+a_2+C \tag{10}
$$
公式（10）$W(a_2)$在其一阶层数为0的位置取得最大值点。因此$\frac{\partial W(a_2)}{\partial a_2}=0$得到的$a_2$为新的$a_2$值
$$
\frac{\partial W(a_2)}{\partial a_2} = -(K_{1,1}+K_{2,2}-2K{1,2})a_2^{new} + K{1,1}\zeta y_2 - K_{1,2}\zeta y_2 + v_1y_2 - v_2y_2 - y_1y_2 +y_2^2 = 0 \tag{11}
$$
公式（11）已经能解出a_2^{new}了，但这样的计算很不方便，可以把公式（11）中不包含$a_2^{new}$的项用$a_2^{old}$来表达。具体方法是将公式(1)(2)(7)代入公式（11）中，并令$\eta = - K_{1,1} - K_{2,2} + 2K_{1,2}$

In [30]:
def K(I, J):
    return I['x'] * J['x'].T

def calc_eta(I, J):
    eta = 2.0 * K(I, J) - K(I, I) - K(J, J)
    if eta >= 0:
        raise Exception('eta>=0')
    return eta

得到最终的一阶导数表达式   

$$
\frac{\partial W(a_2)}{\partial a_2} = \eta a_2^{new} - \eta a_2^{old} + y_2(E_1 - E_2) = 0 \tag{12}
$$
得到  

$$
a_2^{new} = a_2^{old} - \frac{y_2(E_1-E_2)}{\eta} \tag{13}
$$

In [31]:
def update_alpha_j(I, J, C):
    # eta: the optional amount to change alpha[j]
    eta = calc_eta(I, J)
    alphas_j = J['a']-J['y'] *(I['E'] - J['E']) / eta
    # make sure alpha_j is in [0, C]
    L, H = calcLH(C, I, J)   # 此处要用到更新前的J['a']和I['a']
    alphas_j = clipAlpha(alphas_j, H, L)
    J['gap'] = alphas_j - J['a']
    J['a'] = alphas_j
    if(abs(J['gap']) < 0.00001):# the float way to compare
        raise Exception('J not moving enough')

公式（13）计算出来的$a_2^{new}$没有考虑边界，根据公式（6）和（4）可以计算出$a_2^{new}$的边界如图：
![](https://img-blog.csdn.net/20180727105736824?watermark/2/text/aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3UwMTE3MzQxNDQ=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)
当$y_1 \neq y_2$时，
$$\begin{cases}
L = max(0, a_2^{old} - a_1^{old}) \\
H = min(C, C + a_2^{old} - a_1^{old}) \\
\end{cases}
$$
当$y_1 = y_2$时，
$$\begin{cases}
L = max(0, a_2^{old} + a_1^{old} - C) \\
H = min(C, a_2^{old} + a_1^{old}) \\
\end{cases}
$$

In [32]:
def calcLH(C, I, J):
    if (I['y'] != J['y']):
        L = max(0, J['a'] - I['a'])
        H = min(C, C+J['a']-I['a'])
    else:
        L = max(0, I['a'] + J['a'] - C)
        H = min(C, I['a'] + J['a'])
    if L == H:
        raise Exception('L==H')
    return L, H

In [33]:
# 将aj限制在H和L之间
def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if aj < L:
        aj = L
    return aj

更新完$a_2$以后，根据公式2计算出新的$a_1$  
由于$a$的更新不会影响$\zeta y_1$，得出：

$$
a_1^{new} = a_1^{old} + y_1y_2(a_2^{old} - a_2^{new})
$$

In [34]:
def update_alpha_i(I, J):
    I['gap'] = I['y']*J['y']*(-J['gap'])
    I['a'] = I['a'] + I['gap']

更新完一对$a_1$、$a_2$之后，b也要相应的更新  
当 $a_1^{new}$不在边界上，即 $0<a_1^{new}≤C$ ，根据KTT条件可知相应的数据点为支撑向量  
当b_1和b_2都有效的时候它们是相等的，即$b^{new} = b_1^{new} = b_2^{new}$  
但当$a_1$、$a_2$都在边界上且$L \neq H$时，$b_1$、$b_2$之间的值就是和KTT条件一致的阈值。  
SMO选择它们的中点作为新的阈值，即  
$$
b^{new} = \frac{b_1^{new} + b_2^{new}}{2}
$$

In [35]:
def update_b(b, I, J, C):
    b1 = b + calc_b_gap(I, J, I)
    b2 = b + calc_b_gap(I, J, J)
    if (not isInBound(I, C)):
        b = b1
    elif (not isInBound(J, C)):
        b = b2
    else:
        b = (b1 + b2)/2.0
    return b

当$a_1^{new}$不在边界上，即$0 \lt a_1^{new} \le C$，根据KTT条件可知相应的数据点为支撑向量

In [36]:
def isInBound(I, C):
    return not (0 < I['a']) and (I['a'] < C)

如果$a_1^{new}$不在边界上，可知相应的数据点为支撑向量，满足$y_1(w^T+b) = 1$，两边同时乘以$y_1$，得到：  

$$
\sum_{i=1}^{M}a_iy_iK_{i,1}+b=y_1
$$
进而得到$b_1$的值：  

$$
b_1^{new} = y_1 - \sum_{i=3}^{M}a_iy_iK_{i,1} - a_1^{new}y_1K_{1,1} - a_2^{new}y_2K_{2,1} \tag {3}
$$
其中前两项可以写成：  

$$
y_1 - \sum_{i=3}^{M}a_iy_iK_{i,1} = -E_1 + a_1^{old}y_1K_{1,1} + a_2^{old}y_1K_{2,1} + b_old \tag {4}
$$
合并公式（3）、（4）可推出：

$$
b_1^{new} = -E_1 - y_1K_{1,1}(a_1^{new}-a_1^{old}) - y_2K_{2,1}(a_2^{new}-a_2^{old}) + b^{old}
$$

In [37]:
def calc_b_gap(I, J, target):
    return -target['E'] - I['y']*K(I, target)*(I['gap']) - J['y']*K(J, target)*(J['gap'])

In [40]:
import numpy as np

# 使用SMO算法求出alpha，然后根据alpha计算w和b
def smoSimple(dataMatIn, classLabels, C, toler, maxIter):
    dataMat = np.mat(dataMatIn)
    labelMat = np.mat(classLabels).transpose()
    m= np.shape(dataMat)[0]
    alphas = np.mat(np.zeros((m, 1)))# 把所有alpha都初始化为0
    b, iter = 0, 0
    # 迭代matIter次
    while(iter < maxIter):
        alphaPairsChanged = 0
        for i in range(m):
            try: # 可能会有一些情况导致选择的这一对ahpha不能更新
                # 选择一个要更新的alpha
                I = generateI(alphas, labelMat, dataMat, i, b)
                # 选择另一个要更新的alpha
                J = generateJ(alphas, labelMat, dataMat, i, b, m)
                # 更新alpha_i, alpha_j，b也需要相应的更新
                alphas[i], alphas[J['index']], b = update(I, J, C)                                
                alphaPairsChanged += 1
                # print ("iter: %d, i: %d, pairs changed %d" % (iter, i, alphaPairsChanged))
            except UserWarning as err:
                print (err)
        if alphaPairsChanged == 0:
            iter+=1
        else:
            iter = 0
        #print("iteration number: %d" % iter)
    return b, alphas

In [41]:
%%time
b,alphas = smoSimple(dataArr, labelArr, 0.6, 0.001, 40)

TypeError: only size-1 arrays can be converted to Python scalars

In [20]:
b

0

In [21]:
alphas[alphas>0]

matrix([], shape=(1, 0), dtype=float64)

In [22]:
np.shape(alphas[alphas>0])

(1, 0)

In [23]:
for i in range(100):
    if alphas[i]>0.0: print( dataArr[i],labelArr[i])

In [24]:
import matplotlib.pyplot as plt
dataArr, labelArr = svmMLiA.loadDataSet('testSet.txt')
dataMat = np.array(dataArr)
labelMat = np.array(labelArr)
plt.scatter(dataMat[labelMat==1,0], dataMat[labelMat==1,1])
plt.scatter(dataMat[labelMat==-1,0], dataMat[labelMat==-1,1])

#dd = dataMat[alphas>0&labelMat>=1]
w = (0.082216+3.507396)/(3.457096-4.658191)
print (w)
bb = b.getA()[0][0]
print (bb)
x = np.array([0, 10])
y = (-bb - x*w)
plt.plot(x, y)

plt.show()

-2.9886162210316405


AttributeError: 'int' object has no attribute 'getA'